<a href="https://colab.research.google.com/github/hargurjeet/Suprise_Housing/blob/main/Suprise_Housing_Case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the data

In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler

In [89]:
input_url = 'https://raw.githubusercontent.com/hargurjeet/Suprise_Housing/main/train.csv'
raw_data = pd.read_csv(input_url)
raw_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [90]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [91]:
#removing columns with high missing values
pd.set_option('display.max_rows', None)
columns_with_high_missing_values = raw_data.columns[[((raw_data.isnull().sum()/len(raw_data))*100 > 40)]]
columns_with_high_missing_values

/usr/local/lib/python3.8/dist-packages/pandas/core/indexes/base.py:4616: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

In [92]:
print(len(raw_data.columns))
raw_data = raw_data.drop(labels=columns_with_high_missing_values, axis=1)
print(len(raw_data.columns))

81
76


In [93]:
X = raw_data.drop('SalePrice', axis=1)
y = raw_data['SalePrice']

In [94]:
y.shape

(1460,)

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Imputation

In [96]:
## seprating numberical and categorical columns
X_train_numberic = X_train.select_dtypes(include=['number'])
X_train_cat = X_train.select_dtypes(include=['object'])

In [97]:
X_train_numberic = X_train.select_dtypes(include=['number'])
X_train_numberic.shape

(978, 37)

In [98]:
X_train_cat.shape

(978, 38)

In [99]:
X_train_numberic.shape

(978, 37)

In [100]:
X_train_cat.sort_index().head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


In [101]:
pd.set_option('display.max_column', None)

X_train_numberic.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
615,616,85,80.0,8800,6,7,1963,1963,156.0,763,0,173,936,1054,0,0,1054,1,0,1,0,3,1,6,0,1963.0,2,480,120,0,0,0,0,0,0,5,2010
613,614,20,70.0,8402,5,5,2007,2007,0.0,206,0,914,1120,1120,0,0,1120,0,0,1,0,3,1,6,0,NaN,0,0,0,30,0,0,0,0,0,12,2007
1303,1304,20,73.0,8688,7,5,2005,2005,228.0,0,0,1616,1616,1616,0,0,1616,0,0,2,0,3,1,7,0,2005.0,3,834,208,59,0,0,0,0,0,4,2006
486,487,20,79.0,10289,5,7,1965,1965,168.0,836,0,237,1073,1073,0,0,1073,1,0,1,1,3,1,6,0,1965.0,2,515,0,0,0,0,0,0,0,6,2007
561,562,20,77.0,10010,5,5,1974,1975,0.0,1071,123,195,1389,1389,0,0,1389,1,0,1,0,2,1,6,1,1975.0,2,418,240,38,0,0,0,0,0,4,2006


In [102]:
## Observing few column can be classifed as catergorical instead on numberical
cat_cols = ['YearBuilt', 'GarageYrBlt', 'YrSold']
X_train_numberic[cat_cols] = X_train_numberic[cat_cols].astype(object)
X_train_numberic[cat_cols].head()

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,YearBuilt,GarageYrBlt,YrSold
615,1963,1963.0,2010
613,2007,NaN,2007
1303,2005,2005.0,2006
486,1965,1965.0,2007
561,1974,1975.0,2006


In [61]:
X_train_cat.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
615,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,SFoyer,Gable,CompShg,MetalSd,MetalSd,BrkFace,TA,Gd,PConc,TA,TA,Gd,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Abnorml
613,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Feedr,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,NaN,NaN,NaN,Y,New,Partial
1303,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,Unf,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
486,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,MetalSd,MetalSd,BrkFace,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Y,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
561,RL,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,Gable,CompShg,HdBoard,HdBoard,None,TA,TA,CBlock,Gd,TA,Av,ALQ,LwQ,GasA,Gd,Y,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


In [71]:
a = X_train_cat.sort_index().head()
b = X_train_numberic[cat_cols].sort_index().head()

In [103]:
X_train_cat = pd.concat([X_train_cat.sort_index(), X_train_numberic[cat_cols].sort_index()],  axis=1)
X_train_cat.shape

(978, 41)

In [107]:
# X_train_numberic.drop(cat_cols)
X_train_numberic.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
615,616,85,80.0,8800,6,7,1963,1963,156.0,763,0,173,936,1054,0,0,1054,1,0,1,0,3,1,6,0,1963.0,2,480,120,0,0,0,0,0,0,5,2010
613,614,20,70.0,8402,5,5,2007,2007,0.0,206,0,914,1120,1120,0,0,1120,0,0,1,0,3,1,6,0,NaN,0,0,0,30,0,0,0,0,0,12,2007
1303,1304,20,73.0,8688,7,5,2005,2005,228.0,0,0,1616,1616,1616,0,0,1616,0,0,2,0,3,1,7,0,2005.0,3,834,208,59,0,0,0,0,0,4,2006
486,487,20,79.0,10289,5,7,1965,1965,168.0,836,0,237,1073,1073,0,0,1073,1,0,1,1,3,1,6,0,1965.0,2,515,0,0,0,0,0,0,0,6,2007
561,562,20,77.0,10010,5,5,1974,1975,0.0,1071,123,195,1389,1389,0,0,1389,1,0,1,0,2,1,6,1,1975.0,2,418,240,38,0,0,0,0,0,4,2006


In [110]:
X_train_numberic = X_train_numberic.drop(cat_cols, axis=1)
X_train_numberic.shape

(978, 34)

## Imputing numberical columns

In [ ]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_numberic = pd.DataFrame(imp_median.fit_transform(X_train_numberic), columns = X_train_numberic.columns)
X_train_numberic

In [31]:
# scaling the data between 0 and 1
scaler = MinMaxScaler()
X_train_numberic_scaled = pd.DataFrame(scaler.fit_transform(X_train_numberic), columns = X_train_numberic.columns)
X_train_numberic_scaled.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,0.421522,0.382353,0.202055,0.035056,0.555556,0.75,0.659420,0.216667,0.113208,0.135188,...,0.338505,0.140023,0.000000,0.0,0.0,0.0,0.0,0.0,0.363636,1.00
1,0.420151,0.000000,0.167808,0.033195,0.444444,0.50,0.978261,0.950000,0.000000,0.036499,...,0.000000,0.000000,0.054845,0.0,0.0,0.0,0.0,0.0,1.000000,0.25
2,0.893077,0.000000,0.178082,0.034532,0.666667,0.50,0.963768,0.916667,0.165457,0.000000,...,0.588152,0.242707,0.107861,0.0,0.0,0.0,0.0,0.0,0.272727,0.00
3,0.333105,0.000000,0.198630,0.042015,0.444444,0.75,0.673913,0.250000,0.121916,0.148122,...,0.363188,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.454545,0.25
4,0.384510,0.000000,0.191781,0.040711,0.444444,0.50,0.739130,0.416667,0.000000,0.189759,...,0.294781,0.280047,0.069470,0.0,0.0,0.0,0.0,0.0,0.272727,0.00


In [32]:
regressor = RandomForestRegressor(n_estimators=100, max_depth=10)

In [33]:
#===========================================================================
# perform a scikit-learn Recursive Feature Elimination (RFE)
#===========================================================================
# here we want only one final feature, we do this to produce a ranking
n_features_to_select = 1
rfe = RFE(regressor, n_features_to_select=n_features_to_select)
rfe.fit(X_train_numberic_scaled, y_train)

RFE(estimator=RandomForestRegressor(max_depth=10), n_features_to_select=1)

In [34]:
#===========================================================================
# now print out the features in order of ranking
#===========================================================================
feature_list = []
from operator import itemgetter
features = X_train_numberic_scaled.columns.to_list()
for x, y in (sorted(zip(rfe.ranking_ , features), key=itemgetter(0))):
    # print(x, y)
    feature_list.append(y)

In [35]:
top_10_numberic_features = X_train_numberic[feature_list[:10]]

In [25]:
top_10_numberic_features.head()

,OverallQual,GrLivArea,TotalBsmtSF,YearBuilt,BsmtFinSF1,1stFlrSF,GarageArea,LotArea,GarageCars,YearRemodAdd
0,6.0,1054.0,936.0,1963.0,763.0,1054.0,480.0,8800.0,2.0,1963.0
1,5.0,1120.0,1120.0,2007.0,206.0,1120.0,0.0,8402.0,0.0,2007.0
2,7.0,1616.0,1616.0,2005.0,0.0,1616.0,834.0,8688.0,3.0,2005.0
3,5.0,1073.0,1073.0,1965.0,836.0,1073.0,515.0,10289.0,2.0,1965.0
4,5.0,1389.0,1389.0,1974.0,1071.0,1389.0,418.0,10010.0,2.0,1975.0


In [36]:
top_10_numberic_features.head()

,OverallQual,GrLivArea,1stFlrSF,BsmtFinSF1,YearBuilt,TotalBsmtSF,GarageArea,LotArea,YearRemodAdd,GarageCars
0,6.0,1054.0,1054.0,763.0,1963.0,936.0,480.0,8800.0,1963.0,2.0
1,5.0,1120.0,1120.0,206.0,2007.0,1120.0,0.0,8402.0,2007.0,0.0
2,7.0,1616.0,1616.0,0.0,2005.0,1616.0,834.0,8688.0,2005.0,3.0
3,5.0,1073.0,1073.0,836.0,1965.0,1073.0,515.0,10289.0,1965.0,2.0
4,5.0,1389.0,1389.0,1071.0,1974.0,1389.0,418.0,10010.0,1975.0,2.0


In [27]:
# scaling the data between 0 and 1
scaler = MinMaxScaler()
top_10_numberic_features_scaled = pd.DataFrame(scaler.fit_transform(top_10_numberic_features), columns = top_10_numberic_features.columns)
top_10_numberic_features_scaled.head()

,OverallQual,GrLivArea,TotalBsmtSF,YearBuilt,BsmtFinSF1,1stFlrSF,GarageArea,LotArea,GarageCars,YearRemodAdd
0,0.555556,0.135644,0.153191,0.659420,0.135188,0.165213,0.338505,0.035056,0.50,0.216667
1,0.444444,0.148078,0.183306,0.978261,0.036499,0.180358,0.000000,0.033195,0.00,0.950000
2,0.666667,0.241522,0.264484,0.963768,0.000000,0.294172,0.588152,0.034532,0.75,0.916667
3,0.444444,0.139224,0.175614,0.673913,0.148122,0.169573,0.363188,0.042015,0.50,0.250000
4,0.444444,0.198757,0.227332,0.739130,0.189759,0.242084,0.294781,0.040711,0.50,0.416667


## Imputing categorical values

In [29]:
X_train_cat.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,Electrical,KitchenQual,Functional,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
615,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Abnorml
613,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Feedr,Norm,...,SBrkr,TA,Typ,NaN,NaN,NaN,NaN,Y,New,Partial
1303,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,...,SBrkr,Gd,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
486,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal
561,RL,Pave,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,Norm,...,SBrkr,TA,Typ,Attchd,RFn,TA,TA,Y,WD,Normal


In [30]:
X_train_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 978 entries, 615 to 1126
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       978 non-null    object
 1   Street         978 non-null    object
 2   LotShape       978 non-null    object
 3   LandContour    978 non-null    object
 4   Utilities      978 non-null    object
 5   LotConfig      978 non-null    object
 6   LandSlope      978 non-null    object
 7   Neighborhood   978 non-null    object
 8   Condition1     978 non-null    object
 9   Condition2     978 non-null    object
 10  BldgType       978 non-null    object
 11  HouseStyle     978 non-null    object
 12  RoofStyle      978 non-null    object
 13  RoofMatl       978 non-null    object
 14  Exterior1st    978 non-null    object
 15  Exterior2nd    978 non-null    object
 16  MasVnrType     976 non-null    object
 17  ExterQual      978 non-null    object
 18  ExterCond      978 non-null

In [ ]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_numberic = pd.DataFrame(imp_median.fit_transform(X_train_numberic), columns = X_train_numberic.columns)
X_train_numberic